# ipycobe - Interactive Controls with ipywidgets

This example shows how to use ipywidgets sliders, checkboxes, and color pickers to control all aspects of the globe in real-time.

In [ ]:
%pip install ipycobe

In [ ]:
import math
import ipywidgets as widgets
from ipywidgets import (
    FloatSlider,
    IntSlider,
    Checkbox,
    ColorPicker,
    VBox,
    HBox,
    Button,
    FloatText,
    AppLayout,
    HTML,
    Accordion,
    jslink,
    Layout,
)

from ipycobe import Cobe

control_globe = Cobe(
    width=450,
    height=450,
    auto_rotate=True,
    auto_rotate_speed=0.005,
)

control_globe.add_marker((37.7595, -122.4367), size=0.08)  # San Francisco
control_globe.add_marker((51.5074, -0.1278), size=0.08)  # London
control_globe.add_marker((35.6762, 139.6503), size=0.08)  # Tokyo

In [ ]:
phi_slider = FloatSlider(
    value=0.0, min=0.0, max=2 * math.pi, step=0.01, description="Phi:"
)
theta_slider = FloatSlider(
    value=0.0, min=-math.pi / 2, max=math.pi / 2, step=0.01, description="Theta:"
)
auto_rotate_checkbox = Checkbox(value=True, description="Auto Rotate")
rotation_speed_slider = FloatSlider(
    value=0.005,
    min=0.001,
    max=0.05,
    step=0.001,
    description="Speed:",
    readout_format=".3f",
)
draggable_checkbox = Checkbox(value=True, description="Draggable")

In [ ]:
dark_slider = FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description="Dark:")
diffuse_slider = FloatSlider(
    value=1.2, min=0.0, max=3.0, step=0.1, description="Diffuse:"
)
scale_slider = FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description="Scale:")

In [ ]:
map_samples_slider = IntSlider(
    value=16000,
    min=1000,
    max=50000,
    step=1000,
    description="Samples:",
    continuous_update=False,
)
map_brightness_slider = FloatSlider(
    value=6.0, min=1.0, max=12.0, step=0.5, description="Brightness:"
)

In [ ]:
# Link all widgets to the globe
jslink((phi_slider, "value"), (control_globe, "phi"))
jslink((theta_slider, "value"), (control_globe, "theta"))
jslink((auto_rotate_checkbox, "value"), (control_globe, "auto_rotate"))
jslink((rotation_speed_slider, "value"), (control_globe, "auto_rotate_speed"))
jslink((draggable_checkbox, "value"), (control_globe, "draggable"))
jslink((dark_slider, "value"), (control_globe, "dark"))
jslink((diffuse_slider, "value"), (control_globe, "diffuse"))
jslink((scale_slider, "value"), (control_globe, "scale"))
jslink((map_samples_slider, "value"), (control_globe, "map_samples"))
jslink((map_brightness_slider, "value"), (control_globe, "map_brightness"))

In [ ]:
def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip("#")
    return [int(hex_color[i : i + 2], 16) / 255.0 for i in (0, 2, 4)]


def rgb_to_hex(rgb):
    return "#{:02x}{:02x}{:02x}".format(
        int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255)
    )


base_color_picker = ColorPicker(
    value=rgb_to_hex(control_globe.base_color), description="Base:"
)
glow_color_picker = ColorPicker(
    value=rgb_to_hex(control_globe.glow_color), description="Glow:"
)
marker_color_picker = ColorPicker(
    value=rgb_to_hex(control_globe.marker_color), description="Marker:"
)


def on_base_color_change(change):
    control_globe.base_color = hex_to_rgb(change["new"])


def on_glow_color_change(change):
    control_globe.glow_color = hex_to_rgb(change["new"])


def on_marker_color_change(change):
    control_globe.marker_color = hex_to_rgb(change["new"])


base_color_picker.observe(on_base_color_change, names="value")
glow_color_picker.observe(on_glow_color_change, names="value")
marker_color_picker.observe(on_marker_color_change, names="value")

In [ ]:
marker_lat = FloatText(value=0.0, description="Latitude:", step=1.0)
marker_lon = FloatText(value=0.0, description="Longitude:", step=1.0)
marker_size = FloatSlider(value=0.08, min=0.02, max=0.2, step=0.01, description="Size:")
add_marker_btn = Button(description="Add Marker", button_style="primary")
clear_markers_btn = Button(description="Clear All", button_style="warning")


def on_add_marker(btn):
    control_globe.add_marker(
        (marker_lat.value, marker_lon.value), size=marker_size.value
    )


def on_clear_markers(btn):
    control_globe.clear_markers()


add_marker_btn.on_click(on_add_marker)
clear_markers_btn.on_click(on_clear_markers)

In [ ]:
rotation_controls = VBox(
    [
        phi_slider,
        theta_slider,
        auto_rotate_checkbox,
        draggable_checkbox,
        rotation_speed_slider,
    ]
)
lighting_controls = VBox([dark_slider, diffuse_slider, scale_slider])
texture_controls = VBox([map_samples_slider, map_brightness_slider])
color_controls = VBox([base_color_picker, glow_color_picker, marker_color_picker])
marker_controls = VBox(
    [marker_lat, marker_lon, marker_size, HBox([add_marker_btn, clear_markers_btn])]
)

accordion = Accordion(
    children=[
        rotation_controls,
        lighting_controls,
        texture_controls,
        color_controls,
        marker_controls,
    ],
    titles=["Rotation", "Lighting & Scale", "Texture", "Colors", "Markers"],
)
accordion.selected_index = 0

header = HTML(
    value='<h2 style="margin: 0; padding: 10px; background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); color: white; text-align: center;">ipycobe Control Panel</h2>'
)
globe_container = VBox(
    [control_globe],
    layout=Layout(
        display="flex", align_items="center", justify_content="center", padding="10px"
    ),
)

app = AppLayout(
    header=header,
    left_sidebar=None,
    center=globe_container,
    right_sidebar=accordion,
    footer=None,
    pane_widths=["0px", "1fr", "350px"],
    pane_heights=["50px", "1fr", "0px"],
    height="560px",
)

app